In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import precision_score, recall_score
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Corpus de documents

In [3]:
data = pd.read_csv("/content/drive/MyDrive/dataset/bbc_dataset.csv")
data.head()

,Article,Document,Label
0,Broadband challenges TV viewing,The number of Europeans with broadband has exp...,tech
1,Freeze on anti-spam campaign,A campaign by Lycos Europe to target spam-rela...,tech
2,ITunes user sues Apple over iPod,A user of Apple's iTunes music service is suin...,tech
3,A decade of good website design,The web looks very different today than it did...,tech
4,Loyalty cards idea for TV addicts,Viewers could soon be rewarded for watching TV...,tech


In [4]:
data.drop_duplicates(inplace=True)
print(f"il y a {data.shape[0]} documents")
print(data['Label'].value_counts())


il y a 2125 documents
Label
sport            503
business         503
politics         403
entertainment    369
tech             347
Name: count, dtype: int64


### Nettoyage des documents

In [5]:
stemmer = PorterStemmer()
def clean_text(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    words = word_tokenize(text)
    words_stps = [word for word in words if word not in stopwords.words('english')]
    stemmed_words = [stemmer.stem(word) for word in words_stps]
    return " ".join(stemmed_words)

In [6]:
corpus=data.copy()
corpus['Document'] = corpus['Document'].apply(clean_text)
corpus.head()

,Article,Document,Label
0,Broadband challenges TV viewing,number european broadband explod past month we...,tech
1,Freeze on anti-spam campaign,campaign lyco europ target spamrel websit appe...,tech
2,ITunes user sues Apple over iPod,user appl itun music servic su firm say unfair...,tech
3,A decade of good website design,web look differ today year ago back yahoo laun...,tech
4,Loyalty cards idea for TV addicts,viewer could soon reward watch tv loyalti card...,tech


### Indexation TF-IDF

In [7]:
vectorizer = TfidfVectorizer()
term_doc_matrix = vectorizer.fit_transform(corpus['Document'])
tfidf_matrix = term_doc_matrix.toarray()
feature_names = vectorizer.get_feature_names_out()
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=feature_names)
tfidf_matrix.head()

,aa,aaa,aac,aadc,aaliyah,aaltra,aamir,aara,aarhu,aaron,...,zuton,zvonareva,zvyagintsev,âand,âayear,âbn,âbnbn,âm,âmplu,âmâm
0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.07331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Topic Modeling (LSA)

In [8]:
svd = TruncatedSVD(n_components=5, random_state=42)
lsi_matrix = svd.fit_transform(term_doc_matrix)
def get_top_words(svd, feature_names, n_top_words=10):
    topic_words = []
    for topic_idx, topic in enumerate(svd.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topic_words.append(top_words)
    return topic_words
topic_words = get_top_words(svd, feature_names)
for topic_idx, topic in enumerate(topic_words):
    print(f"Topic {topic_idx + 1}: {', '.join(topic)}")

Topic 1: said, mr, would, year, game, peopl, labour, us, govern, elect
Topic 2: mr, labour, elect, parti, blair, tax, tori, brown, govern, minist
Topic 3: labour, win, elect, england, blair, play, parti, game, mr, tori
Topic 4: film, award, best, nomin, oscar, actor, star, actress, music, festiv
Topic 5: mobil, phone, use, technolog, peopl, game, user, servic, comput, music


### Requête


1.   Analyse de la requête
2.   Détection des topics
3.   Documents pertinents



In [9]:
def find_similar_docs(query_vector, lsi_matrix):
    similarities = cosine_similarity(query_vector, lsi_matrix).flatten()
    most_similar_indices = similarities.argsort()[:-11:-1]
    return most_similar_indices,similarities

In [10]:
#sanstopicmodeling
def find_similar_docssans(query_vector, tfidf_matrix):
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    most_similar_indices = similarities.argsort()[:-11:-1]
    return most_similar_indices,similarities

In [29]:
query_text="Microsoft"
clean_text(query_text)
query_vector = vectorizer.transform([query_text])
query_vector = svd.transform(query_vector)
similar_docs,similarities = find_similar_docs(query_vector, lsi_matrix)
print(f"Requête : {query_text}")
for doc_index in similar_docs:
         print(f"  Document {doc_index} : {similarities[doc_index]} : {data['Document'].iloc[doc_index]}")

Requête : Microsoft
  Document 234 : 0.9785986828549649 : Security firms are warning about several mobile phone viruses that can spread much faster than similar bugs. The new strains of the Cabir mobile phone virus use short-range radio technology to leap to any vulnerable phone as soon as it is in range. The Cabir virus only affects high-end handsets running the Symbian Series 60 phone operating system. Despite the warnings, there are so far no reports of any phones being infected by the new variants of Cabir. The original Cabir worm came to light in mid-June 2004 when it was sent to anti-virus firms as a proof-of-concept program. A mistake in the way the original Cabir was written meant that even if it escaped from the laboratory, the bug would only have been able to infect one phone at a time. However, the new Cabir strains have this mistake corrected and will spread via short range Bluetooth technology to any vulnerable phone in range. Bluetooth has an effective range of a few tens

In [19]:
query_text="Microsoft"
clean_text(query_text)
query_vector = vectorizer.transform([query_text])
similar_docs,similarities = find_similar_docssans(query_vector, tfidf_matrix)
print(f"Requête : {query_text}")
for doc_index in similar_docs:
         print(f"  Document {doc_index} : {similarities[doc_index]} : {data['Document'].iloc[doc_index]}")

Requête : Microsoft
  Document 222 : 0.4700536329219828 : Microsoft is investigating a trojan program that attempts to switch off the firm's anti-spyware software. The spyware tool was only released by Microsoft in the last few weeks and has been downloaded by six million people. Stephen Toulouse, a security manager at Microsoft, said the malicious program was called Bankash-A Trojan and was being sent as an e-mail attachment. Microsoft said it did not believe the program was widespread and recommended users to use an anti-virus program. The program attempts to disable or delete Microsoft's anti-spyware tool and suppress warning messages given to users. It may also try to steal online banking passwords or other personal information by tracking users' keystrokes. Microsoft said in a statement it is investigating what it called a criminal attack on its software. Earlier this week, Microsoft said it would buy anti-virus software maker Sybari Software to improve its security in its Windows

### Implémentation du moteur de recherche

In [ ]:
def find_similar_docs_2(query_vector, lsi_matrix):
    similarities = cosine_similarity(query_vector, lsi_matrix).flatten()
    most_similar_indices = similarities.argsort()[::-1]
    return most_similar_indices,similarities

In [ ]:
!pip install flask pyngrok

In [ ]:
!pip install flask-ngrok

In [ ]:
file='/content/drive/MyDrive/templates'
files='/content/drive/MyDrive/static'

In [ ]:
def query_process(query):
  query_text=clean_text(query)
  query_vector = vectorizer.transform([query_text])
  query_vector = svd.transform(query_vector)
  return query_vector

In [ ]:
from flask import Flask, render_template,request
from pyngrok import ngrok,conf
from flask_ngrok import run_with_ngrok
conf.get_default().auth_token = "2qRwqyoL3XKciS6OYqN8Z76Vjpo_2a1uTCtmKHUNkaMVd7Hbq"

In [ ]:
app = Flask(__name__,template_folder=file,static_folder=files)
run_with_ngrok(app)
@app.route('/')
def home():
    return render_template("index_search.html")
@app.route('/search',methods=['POST'])
def search():
  if request.method == 'POST':
    query = request.form['query']
    query_vector=query_process(query)
    similar_docs,similarities = find_similar_docs_2(query_vector, lsi_matrix)
    result_dict={}
    doc_number=0
    for doc_index in similar_docs:
      if similarities[doc_index]>0.88:
        doc_number+=1
        result_dict[doc_index] = {
        'article': data['Article'].iloc[doc_index],
        'document': data['Document'].iloc[doc_index]
         }
    return render_template("index_search.html", results=result_dict,requete=query,results_number=doc_number)
  return render_template("index_search.html")

In [ ]:
import time
if __name__ == '__main__':
    ngrok.connect(5000)
    time.sleep(5)
    app.run()

